# YOLOv8 Hailo Compilation Guide

This notebook demonstrates how to compile an onnx model file for Hailo devices using the Hailo Dataflow Compiler (DFC).

## Prerequisites

You need to download the following files from the [Hailo Developer Zone](https://hailo.ai/developer-zone/software-downloads/):
1.  **Hailo Dataflow Compiler (DFC)** Python wheel (e.g., `hailo_dataflow_compiler-*.whl`)
2.  **Hailo Model Zoo** Python wheel (e.g., `hailo_model_zoo-*.whl`)

**Note:** Ensure the DFC version supports the Python version of this Colab environment (currently likely Python 3.10 or 3.11). Check `!python --version`.

In addition, trained onnx file (from training stage) is required.

In [ ]:
!python --version

Python 3.12.12


In [ ]:
from google.colab import files
import os

print("Please upload the Hailo DFC & Model Zoo wheels, and the trained model onnx file:")
uploaded = files.upload()

dfc_whl = 'dfc'  # [f for f in uploaded.keys() if "hailo_dataflow_compiler" in f]
hmz_whl = [f for f in uploaded.keys() if "hailo_model_zoo" in f]
onnx = [f for f in uploaded.keys() if "onnx" in f]

if not dfc_whl or not hmz_whl or not onnx:
    raise FileNotFoundError("Hailo DFC or Model Zoo wheels or onnx model not uploaded. Please upload them to proceed.")
else:
    print(f"Found DFC: {dfc_whl[0]}")
    print(f"Found Model Zoo: {hmz_whl[0]}")
    print(f"onnx: {onnx[0]}")

In [ ]:
!apt-get install -y graphviz graphviz-dev pkg-config
!pip install pygraphviz

In [ ]:
# Install Hailo wheels
if dfc_whl and hmz_whl:
    !pip install "{dfc_whl[0]}" "{hmz_whl[0]}"

In [ ]:
!pip install kagglehub

In [ ]:
import kagglehub
import shutil
import os

# Download latest version
path = kagglehub.dataset_download("kushagrapandya/barcode-detection")

print("Path to dataset files:", path)

# Move to a convenient location
dataset_dir = "/content/barcode-detect"
if not os.path.exists(dataset_dir):
    shutil.copytree(path, dataset_dir)
    print(f"Dataset moved to {dataset_dir}")
else:
    print(f"Dataset already exists at {dataset_dir}")

In [ ]:
!git clone https://github.com/hailo-ai/hailo_model_zoo.git

In [ ]:
!hailomz compile \
    --ckpt /content/barcode-detection.onnx \
    --calib-path /content/barcode-detect/valid/ \
    --yaml /content/hailo_model_zoo/hailo_model_zoo/cfg/networks/yolov8s.yaml \
    --classes 2 \
    --hw-arch hailo10h \
    --performance